# Creating command-line Tools

In [1]:
from ctapipe.core import Tool, Component, TelescopeComponent
from ctapipe.core.traits import (
    Integer, Float, List, Dict, Unicode,
    TraitError, observe, FloatTelescopeParameter,
    Path,
)
import logging
from time import sleep
from astropy import units as u
from ctapipe.utils import get_dataset_path

In [2]:
GAMMA_FILE = get_dataset_path('gamma_test.simtel.gz')

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [3]:
class MyComponent(Component):
    """ A Component that does stuff """

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")


# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Path(
        help="input file name", 
        exists=None,  # set to True to require existing, False for requiring non-existing
        directory_ok=False,
    ).tag(config=True)
    outfile = Path(
        help="output file name",
        exists=False, directory_ok=False
    ).tag(config=True)

    @observe("outfile")
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class TelescopeWiseComponent(TelescopeComponent):
    """ a component that contains parameters that are per-telescope configurable """

    param = FloatTelescopeParameter(
        help="Something configurable with telescope patterns", default_value=5.0
    ).tag(config=True)

In [4]:
MyComponent()

value,-1,Value to use (default: -1)


In [5]:
AdvancedComponent()

infile,None,input file name (default: None)
outfile,None,output file name (default: None)
value1,-1,Value to use (default: -1)


`TelescopeComponents` need to have a subarray given to them in order to work (since they need one to turn a `TelescopeParameter` into a concrete list of values for each telescope.  Here we will give a dummy one:

In [6]:
from ctapipe.instrument import SubarrayDescription, TelescopeDescription

subarray = SubarrayDescription(
    "Junk",
    tel_positions={1: (0.0, 0.0, 0.0) * u.m, 2: (1.0, 1.0, 0.0) * u.m},
    tel_descriptions={
        1: TelescopeDescription.from_name("LST", "LSTCam"),
        2: TelescopeDescription.from_name("MST", "NectarCam"),
    },
)
subarray.info()

Resorting to default CameraReadout, File does not exist: (LSTCam.camreadout)


Resorting to default CameraReadout, File does not exist: (NectarCam.camreadout)


Subarray : Junk
Num Tels : 2
Footprint: 0.00 km2

                TYPE  Num IDmin  IDmax
      LST_LST_LSTCam    1    1 ..   1
   MST_MST_NectarCam    1    2 ..   2


In [7]:
TelescopeWiseComponent(subarray=subarray)

param,"[('type', '*', 5.0)]",Something configurable with telescope patterns (default: traitlets.Undefined)


This TelescopeParameters can then be set using a list of patterns like:
```python
component.param = [ 
    ("type", "LST*",3.0), 
    ("type", "MST*", 2.0), 
    (id, 25, 4.0) 
]
```

These get translated into per-telescope-id values once the subarray is registered.  After that one acccess the per-telescope id values via:
```python
component.param.tel[tel_id]
```

## Now create an executable Tool that contains the Components

In [8]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = dict(
        infile='AdvancedComponent.infile',
        iterations='MyTool.iterations'
    )

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent, TelescopeWiseComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup_comp(self):
        # when constructing Components, you must add them to the 
        # list of registered instances using add_component. This allows
        # the full configuration to be tracked
        self.comp = self.add_component(MyComponent(parent=self))
        self.comp2 = self.add_component(SecondaryMyComponent(parent=self))

    def setup_advanced(self):
        self.advanced = self.add_component(AdvancedComponent(parent=self))
        self.comp3 = self.add_component(TelescopeWiseComponent(parent=self, subarray=subarray))


    def setup(self):
        self.setup_comp()
        self.setup_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [9]:
tool=MyTool()

In [10]:
tool.print_help()

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--infile=<Path> (AdvancedComponent.infile)
    Default: None
    input file name
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--config=<Path> (Tool.config_file)
    Default: None
    name of a configuration file with parameters to load in addition to command-
    line parameters

To see all available configurables, use `--help-all`



The following  is equivalant to the user typing `mytool --help-all`

In [11]:
tool.print_help(classes=True)

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--infile=<Path> (AdvancedComponent.infile)
    Default: None
    input file name
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--config=<Path> (Tool.config_file)
    Default: None
    name of a configuration file with parameters to load in addition to command-
    line parameters

Class parameters
----------------

Parameters are set from command-line arguments of the form:
`--Class.trait=value`. This line is evaluated in Python, so simple expressions
are allowed, e.g.:: `--C.a='range(3)'` For setting C.a=[0,1,2].

MyTool options
--------------


## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile gamma_test.simtel.gz --iterations=3
```

As Tools are intended to be exectutables, they are raising `SystemExit` on exit.
Here, we use them to demonstrate how it would work, so we catch the `SystemExit`.

In [12]:
try:
    tool.run(argv=['--infile',  GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

INFO [MyTool] (tool/initialize): ctapipe version 0.8.0.post10+git262b6aa


INFO [MyTool] (tool/run): Starting: mytool


INFO [MyTool] (<ipython-input-8-24ed1862db7e>/start): Performing 5 iterations...


INFO [MyTool] (<ipython-input-8-24ed1862db7e>/start): ITERATION 0


INFO [MyTool] (<ipython-input-8-24ed1862db7e>/start): ITERATION 1


...

INFO [MyTool] (<ipython-input-8-24ed1862db7e>/start): ITERATION 3


INFO [MyTool] (<ipython-input-8-24ed1862db7e>/start): ITERATION 4


WARNING [MyTool] (<ipython-input-8-24ed1862db7e>/finish): Shutting down.


INFO [MyTool] (tool/run): Finished: mytool


INFO [MyTool] (tool/run): Output: 


In [13]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 


try:
    tool.run(argv=['--log-level','INFO','--infile', GAMMA_FILE, '--iterations','3'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2020-07-05 11:43:47 : INFO [MyTool initialize] ctapipe version 0.8.0.post10+git262b6aa


2020-07-05 11:43:47 : INFO [MyTool run] Starting: mytool


2020-07-05 11:43:47 : INFO [MyTool start] Performing 3 iterations...


2020-07-05 11:43:47 : INFO [MyTool start] ITERATION 0


2020-07-05 11:43:48 : INFO [MyTool start] ITERATION 1


2020-07-05 11:43:48 : INFO [MyTool start] ITERATION 2


2020-07-05 11:43:48 : WARNING [MyTool finish] Shutting down.


2020-07-05 11:43:48 : INFO [MyTool run] Finished: mytool


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


here we change the log-level to DEBUG:

In [14]:
try:
    tool.run(argv=['--log-level','DEBUG','--infile', GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2020-07-05 11:43:48 : INFO [MyTool initialize] ctapipe version 0.8.0.post10+git262b6aa


2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] Config changed:


2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] {'AdvancedComponent': {'infile': '/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'}, 'attach_subarray': <traitlets.config.loader.LazyConfigValue object at 0x7ff6cbfec550>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2020-07-05 11:43:48 : INFO [MyTool run] Starting: mytool


2020-07-05 11:43:48 : DEBUG [MyTool run] CONFIG: {'MyTool': {'config_file': None, 'iterations': 3, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s', 'log_level': 10}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': PosixPath('/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'), 'outfile': None, 'value1': -1}, 'TelescopeWiseComponent': {'param': [('type', '*', 5.0)]}}


...

2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : DEBUG [MyTool run] PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "19a8f593-086d-4dfe-ba8b-1011534e7726",
      "start": {
         "time_utc": "2020-07-05T11:43:47.377"
      },
      "stop": {
         "time_utc": "2020-07-05T11:43:47.901"
      },
      "system": {
         "ctapipe_version": "0.8.0.post10+git262b6aa",
         "ctapipe_resources_version": "0.3.0",
         "eventio_version": "1.4.1",
         "ctapipe_svc_path": null,
         "executable": "/home/travis/virtualenv/python3.7.6/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "travis-job-dac37d3b-4b29-49d5-812a-4e9dcbe13e8e",
            "version": "#32-Ubuntu SMP Tue Feb 11 03:55:48 UTC 2020",
            "system": "Linux",
            "release": "5.0.0-1031-gcp",
            "libcver": [
               "glibc"

2020-07-05 11:43:48 : DEBUG [MyTool exit] Exiting application: mytool


you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [15]:
tool.iterations = 1
tool.log_level = 0

try:
    tool.run(['--infile', GAMMA_FILE])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] Config changed:


2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] {'AdvancedComponent': {'infile': '/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'}, 'attach_subarray': <traitlets.config.loader.LazyConfigValue object at 0x7ff6c77fbdd0>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2020-07-05 11:43:48 : INFO [MyTool initialize] ctapipe version 0.8.0.post10+git262b6aa


2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] Config changed:


2020-07-05 11:43:48 : DEBUG [MyTool _config_changed] {'AdvancedComponent': {'infile': '/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'}, 'attach_subarray': <traitlets.config.loader.LazyConfigValue object at 0x7ff6c7870fd0>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


...

2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : INFO [MyTool run] Output: 


2020-07-05 11:43:48 : DEBUG [MyTool run] PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "19a8f593-086d-4dfe-ba8b-1011534e7726",
      "start": {
         "time_utc": "2020-07-05T11:43:47.377"
      },
      "stop": {
         "time_utc": "2020-07-05T11:43:47.901"
      },
      "system": {
         "ctapipe_version": "0.8.0.post10+git262b6aa",
         "ctapipe_resources_version": "0.3.0",
         "eventio_version": "1.4.1",
         "ctapipe_svc_path": null,
         "executable": "/home/travis/virtualenv/python3.7.6/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "travis-job-dac37d3b-4b29-49d5-812a-4e9dcbe13e8e",
            "version": "#32-Ubuntu SMP Tue Feb 11 03:55:48 UTC 2020",
            "system": "Linux",
            "release": "5.0.0-1031-gcp",
            "libcver": [
               "glibc"

2020-07-05 11:43:48 : DEBUG [MyTool exit] Exiting application: mytool


see what happens when a value is set that is not of the correct type:

In [16]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

bad value: The 'iterations' trait of a MyTool instance must be an int, but a value of 'badval' <class 'str'> was specified.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [17]:
tool.advanced.outfile = "Another.txt"

2020-07-05 11:43:48 : WARNING [MyTool.AdvancedComponent on_outfile_changed] Outfile was changed to '{'name': 'outfile', 'old': None, 'new': PosixPath('/home/travis/build/cta-observatory/ctapipe/docs/examples/Another.txt'), 'owner': <__main__.AdvancedComponent object at 0x7ff6c77fbf10>, 'type': 'change'}'


we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [18]:
tool2 = MyTool()

In [19]:
try:
    tool2.run(argv=['--config', 'Tools.json'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2020-07-05 11:43:49 : INFO [MyTool initialize] ctapipe version 0.8.0.post10+git262b6aa


2020-07-05 11:43:49 : INFO [MyTool run] Starting: mytool


2020-07-05 11:43:49 : DEBUG [MyTool run] CONFIG: {'MyTool': {'config_file': PosixPath('/home/travis/build/cta-observatory/ctapipe/docs/examples/Tools.json'), 'iterations': 5, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s', 'log_level': 10}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': PosixPath('/home/travis/build/cta-observatory/ctapipe/docs/examples/something.txt'), 'outfile': None, 'value1': -1}, 'TelescopeWiseComponent': {'param': [('type', '*', 5.0)]}}


2020-07-05 11:43:49 : INFO [MyTool start] Performing 5 iterations...


2020-07-05 11:43:49 : INFO [MyTool start] ITERATION 0


...

2020-07-05 11:43:49 : INFO [MyTool run] Output: 


2020-07-05 11:43:49 : INFO [MyTool run] Output: 


2020-07-05 11:43:49 : INFO [MyTool run] Output: 


2020-07-05 11:43:49 : DEBUG [MyTool run] PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "19a8f593-086d-4dfe-ba8b-1011534e7726",
      "start": {
         "time_utc": "2020-07-05T11:43:47.377"
      },
      "stop": {
         "time_utc": "2020-07-05T11:43:47.901"
      },
      "system": {
         "ctapipe_version": "0.8.0.post10+git262b6aa",
         "ctapipe_resources_version": "0.3.0",
         "eventio_version": "1.4.1",
         "ctapipe_svc_path": null,
         "executable": "/home/travis/virtualenv/python3.7.6/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "travis-job-dac37d3b-4b29-49d5-812a-4e9dcbe13e8e",
            "version": "#32-Ubuntu SMP Tue Feb 11 03:55:48 UTC 2020",
            "system": "Linux",
            "release": "5.0.0-1031-gcp",
            "libcver": [
               "glibc"

2020-07-05 11:43:49 : DEBUG [MyTool exit] Exiting application: mytool


In [20]:
print(tool2.advanced.infile)

/home/travis/build/cta-observatory/ctapipe/docs/examples/something.txt


In [21]:
print(tool2.config)

{'MyTool': {'config_file': 'Tools.json', 'log_level': 'DEBUG'}, 'AdvancedComponent': {'infile': 'something.txt'}, 'attach_subarray': <traitlets.config.loader.LazyConfigValue object at 0x7ff6c7808d50>}


In [22]:
tool2.is_setup

True

In [23]:
tool3 = MyTool()

In [24]:
tool3.is_setup

False

In [25]:
tool3.initialize(argv=[])

2020-07-05 11:43:49 : INFO [MyTool initialize] ctapipe version 0.8.0.post10+git262b6aa


In [26]:
tool3.is_setup

False

In [27]:
tool3

config_file,None,name of a configuration file with parameters to load in addition to command-line parameters (default: None)
iterations,5,Number of times to run (default: 5)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_format,%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s,The Logging format template (default: %(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s)
log_level,20,Set the log level by value or name. (default: 30)


In [28]:
tool

config_file,None,name of a configuration file with parameters to load in addition to command-line parameters (default: None)
iterations,3,Number of times to run (default: 5)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_format,%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s,The Logging format template (default: %(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s)
log_level,10,Set the log level by value or name. (default: 30)


In [29]:
tool.comp2

value,-1,Value to use (default: -1)


## Getting the configuration of an instance

In [30]:
tool.get_current_config()

{'MyTool': {'config_file': None,
  'iterations': 3,
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 10},
 'MyComponent': {'value': -1},
 'SecondaryMyComponent': {'value': -1},
 'AdvancedComponent': {'infile': PosixPath('/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'),
  'outfile': PosixPath('/home/travis/build/cta-observatory/ctapipe/docs/examples/Another.txt'),
  'value1': -1},
 'TelescopeWiseComponent': {'param': [('type', '*', 5.0)]}}

In [31]:
tool.iterations = 12
tool.get_current_config()

{'MyTool': {'config_file': None,
  'iterations': 12,
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 10},
 'MyComponent': {'value': -1},
 'SecondaryMyComponent': {'value': -1},
 'AdvancedComponent': {'infile': PosixPath('/home/travis/virtualenv/python3.7.6/lib/python3.7/site-packages/ctapipe_resources/gamma_test.simtel.gz'),
  'outfile': PosixPath('/home/travis/build/cta-observatory/ctapipe/docs/examples/Another.txt'),
  'value1': -1},
 'TelescopeWiseComponent': {'param': [('type', '*', 5.0)]}}

## Writing a Sample Config File

In [32]:
print(tool.generate_config_file())

# Configuration file for mytool.

#------------------------------------------------------------------------------
# Application(SingletonConfigurable) configuration
#------------------------------------------------------------------------------

## This is an application.

## The date format used by logging formatters for %(asctime)s
#c.Application.log_datefmt = '%Y-%m-%d %H:%M:%S'

## The Logging format template
#c.Application.log_format = '[%(name)s]%(highlevel)s %(message)s'

## Set the log level by value or name.
#c.Application.log_level = 30

#------------------------------------------------------------------------------
# Tool(Application) configuration
#------------------------------------------------------------------------------

## This is an application.

## name of a configuration file with parameters to load in addition to command-
#  line parameters
#c.Tool.config_file = None

## The Logging format template
#c.Tool.log_format = '%(levelname)s [%(name)s] (%(module)s/%(func